In [8]:
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbn
import sklearn as skl
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
#from keras.models import Sequential
#from keras.layers import Dense
#import keras
#from keras.models import Sequential


I_nodes = 12
H_nodes = 100 #1 layer with 100 neurons
O_nodes = 1 # Equal to a number of classes
Max_Iters = 100000
L_Rate = 0.001


N_Trees = 100
M_leaf = [1, 10]

KFolds = 10
ANN_error_tot = np.empty(2)
RF_error_tot = np.empty(2)


cvhl = np.array([25, 100, 500]) #Cross Validation Hidden layers
cvnt = np.array([10, 50, 100]) #Cross Validation Number of Trees

def summary(input_d):
    mean = input_d.mean()
    std = input_d.std()
    minim = input_d.min()
    maxim = input_d.max()
    
    cols = input_d.shape[1] #len(input_d.columns)
    rows = input_d.shape[0]
    dty = input_d.dtypes

    #print("Means:")
    #print(mean)
    #print("stds:")
    print("Data Summary:")
    #print(input_d.describe(include = 'all'))
    desc = input_d.describe().transpose()
    #print(desc)
    print("Empty values: {}.".format(input_d.isnull().values.any()))
    print("Shape of the data: columns = {}, rows = {}".format(cols, rows))
    #print(input_d.select_dtypes(exclude=["number","bool_","object_"]))
    #nuclear_inp_df.select_dtypes(include=["category"])
    #summar = pd.DataFrame(mean, std, minim, maxim)
    #print(mean, std, minim, maxim)
    #print(cols, rows)
    #print(input_d.dtypes)
    cols2 = input_d.columns
    num_cols = input_d._get_numeric_data().columns
    print ("Categorical Features: {}".format(list(set(cols2) - set(num_cols))))
    return cols, rows

def visualise(boxd, dens):
    #nuclear_inp_df.groupby(["Status"]).size()
    #t2 = t.groupby(["Status"])
    #print(t2)
    #t.plot.box(by='Status')
    #plt.set_title('Boxplot Status')
    #t2.plot.box()
    #plt.plot(stat, power_sens_1)
    #t.groupby('Status').size().plot(kind='bar')
    #nuclear_inp_df.boxplot(column="Status")
    #t.plot(x='Status',y = 'Power_range_sensor_1', kind='box')
    #plt.xlabel("test")
    #nuclear_inp_df.boxplot(column="Status")
    
    plt.figure()
    #p1 = plt.subplot(211)
    bplot = boxd.boxplot(by = 'Status')
    plt.title("Boxplot of Power sensor data grouped by status")
    plt.suptitle("")
    bplot.set_xlabel("Status of Reactor")
    bplot.set_ylabel("Power Range Sensor 1")
    plt.show()
    
    
    plt.figure()
    #p2 = plt.subplot(212)
    abnorm = dens[dens.Status == "Abnormal"]
    norm = dens[dens.Status == "Normal"]
    plt.title("Density plot of Pressure Sensor data grouped by status")
    dplot = sbn.kdeplot(abnorm['Pressure _sensor_1'], shade=True, label="Abnormal")
    dplot = sbn.kdeplot(norm['Pressure _sensor_1'], shade=True, label="Normal")
    dplot.set_xlabel("Pressure Range Sensor 1")
    dplot.set_ylabel("Density")
    plt.show()
    
    #plt.hist(press_sens_1, density = True, histtype = 'step')
    #press_sens_1_df = pd.DataFrame(press_sens_1)
    #press_sens_1.plot.kde()
    #for ti, gr in dens.groupby('Status'):
    #dplot = dens.plot.density()
    #dplot = dens.hist(by = 'Status', sharex=True, sharey=True)
    #for d in dplot.flatten():
        #d.set_xlabel("Pressure Range Sensor 1")
        #d.set_ylabel("Density")
    #dplot = dens.groupby('Status').plot.density(subplots = True)
    #grouping = dens.groupby('Status')#, axis = 1)#.size().plot(kind='bar')
    #new = [grouping.get_group(t) for t in grouping.groups]
    #print(new)
    #print(grouping.groups)
    #dplot1 = grouping.get_group('Abnormal')
    #abnorm = dplot1.iloc[:,1]
    #print(dplot1)
    #dplot2 = grouping.get_group('Normal')
    #norm = dplot2.iloc[:, 1]
    #dplotfinal = pd.DataFrame(abnorm, norm, columns = ['Abnormal', 'Normal'])
    #print (dplotfinal)
    #dplotfinal.plot.density()
    #abnorm.plot.density()
    #dplot2.plot.density()
    #for stat, gr in grouping:
    #gr.plot('Status', title=stat)
    #nuclear_inp_df.plot.box(grid='True')
    #plt.boxplot(x = power_sens_1)
    #plt.boxplot(test)
    
def randomsplit(input_d): #Randomises which part of the input file will be used for training or testing
    
    #First convert Status from string to int/bool
    input_d['Status'] = (input_d['Status'] != 'Abnormal').astype(int)
    
    #Then Split data 90:10 for training and test
    length = len(input_d)
    output = input_d.sample(frac=1)
    ninety = int(round(0.9*(length)))
    output_train = output[0:ninety]
    output_test = output[ninety:length]
    return output_train, output_test    

class artneurnet:
    #Sigmoid: 1/1+e^-z = 1/1+e^(- sum(wj*xj-b))
    def __init__(self, input_nodes, output_nodes, maximum_iterations, learning_rate):
        print("-- Initialising ANN --")
        self.inpn = input_nodes
        self.outn = output_nodes
        self.maxi = maximum_iterations
        self.lear = learning_rate
        
    def process(self, train, test, hidden_nodes, disp, total_accuracy):
        if (disp == True):
            print("Artificial Neural network: ")
        network = MLPClassifier(hidden_layer_sizes = hidden_nodes, activation = 'logistic', learning_rate_init = self.lear, max_iter = self.maxi)
        train_labels = train.iloc[:,0]
        train_samp = pd.DataFrame(train.iloc[:,1:13])
        test_labels = test.iloc[:,0]
        test_samp = pd.DataFrame(test.iloc[:,1:13])
        network.fit(train_samp, train_labels)
        train_prediction =  network.predict(train_samp)
        test_prediction = network.predict(test_samp)
        #print(network.score(test_labels,test_prediction))
        train_acc = accuracy_score(train_labels, train_prediction)
        test_acc = accuracy_score(test_labels, test_prediction)
        conf_mat = confusion_matrix(test_labels, test_prediction)
        if (disp == True):
            print("Number of hidden nodes: {}".format(hidden_nodes))
            print("Training Accuracy: {}".format(train_acc))
            print("Testing Accuracy: {}".format(test_acc))
            print("Confusion Matrix: {} , {}".format(conf_mat[0], conf_mat[1]))
            return total_accuracy
        else:
            total_accuracy[0] = total_accuracy[0] + train_acc
            total_accuracy[1] = total_accuracy[1] + test_acc
            return total_accuracy
            
class randforests:
    def __init__(self):#, estim):
        print("-- Initialising Random Forest --")
        self.meanacc = []
        #self.num_estim = estim
        
    def classify(self, minsamp, train, test, num_estim, disp, totacc):
        if (disp == True):
            print("Random Forest Classifier: ")
        forest = RandomForestClassifier(n_estimators=num_estim, min_samples_leaf = minsamp)
        #print(train)
        train_labels = train.iloc[:,0]
        train_samp = pd.DataFrame(train.iloc[:,1:13])
        test_labels = test.iloc[:,0]
        test_samp = pd.DataFrame(test.iloc[:,1:13])
        forest.fit(train_samp, train_labels)
        importance = forest.feature_importances_
        train_prediction =  forest.predict(train_samp)
        test_prediction  = forest.predict(test_samp)
        #print(prediction)
        #forest.score(prediction, test_labels)
        train_acc = accuracy_score(train_labels, train_prediction)
        test_acc = accuracy_score(test_labels, test_prediction)
        conf_mat = confusion_matrix(test_labels, test_prediction)
        #tn, fp, fn, tp = confusion_matrix(test_labels, test_prediction)
        if (disp == True):
            print("Min number of samples for leaf node: {}".format(minsamp))
            print("Training Accuracy: {}".format(train_acc))
            print("Testing Accuracy: {}".format(test_acc))
            print("Confusion Matrix: {} , {}".format(conf_mat[0], conf_mat[1]))
            #print("Importance of each feature: {}".format(importance))
            #print(tn, fp, fn, tp)
            return totacc
        else:
            #self.meanacc.append(test_acc)
            totacc[0] = totacc[0] + train_acc
            totacc[1] = totacc[1] + test_acc
            return totacc
        #print(self.meanacc)
        
#def CrossVal(folds, inp, i):
    #kf =  KFold(n_splits=folds, shuffle = True
    #for traini, testi in kf.split(inp):
        #print(traini)
        #train = pd.DataFrame(inp, traini)
        
        #test = pd.DataFrame(inp, testi)
        #ann1.process(tr, te, cvhl[i], False)
        #print("---------------------")
        #rfc1.classify(10, tr, te,cvnt[i], False)
        #print("---------------------"))
        #return train,test
        #train, test = inp[traini], inp[testi]
        #print(len(testi))
        
#Importing
nuclear_inp = pd.read_csv('CMP3744M_ADM_Assignment 2-dataset-nuclear_plants.csv', index_col=None)
#pd.read_csv('CMP3744M_ADM_Assignment 2-dataset-nuclear_plants.csv', index_col=None)
nuclear_inp_df = pd.DataFrame(nuclear_inp)

#Section 1: Summary
(c,r) = summary(nuclear_inp_df)

#Section 1: Visualising
statpow1 = pd.DataFrame(nuclear_inp_df.iloc[:,0:2])# columns = nuclear_inp_df.groupby(["Status"]))
statpress1 = nuclear_inp_df[['Status', 'Pressure _sensor_1']].copy()
#visualise(statpow1, statpress1)

#Section 3: Splitting Data
(out_tr, out_te) = randomsplit(nuclear_inp_df)
#print(out_tr)

#Section 3: ANN
ann1 = artneurnet(r, O_nodes, Max_Iters, L_Rate)
ANN_error_tot = ann1.process(out_tr, out_te, H_nodes, True, ANN_error_tot)

#Section 3: Random Forests
rfc1 = randforests()#N_Trees)
#ANN_error_tot = 0
RF_error_tot = rfc1.classify(1, out_tr, out_te, N_Trees, True, RF_error_tot)
print("---------------------")
RF_error_tot = rfc1.classify(10, out_tr, out_te, N_Trees, True, RF_error_tot)

print("---------------------")
#Section 4: Cross Validation
print("Cross Validation")
print("---------------------")

    #(tr, te) = CrossVal(KFolds, nuclear_inp_df, i)
kf =  KFold(n_splits=KFolds, shuffle = True)
for i in range (cvhl.size):
    for traini, testi in kf.split(nuclear_inp_df):
        #print(traini)
        train = pd.DataFrame(nuclear_inp_df, traini)
        test = pd.DataFrame(nuclear_inp_df, testi)

        ANN_error_tot = ann1.process(train, test, cvhl[i], False, ANN_error_tot)
        #print("---------------------")
        RF_error_tot = rfc1.classify(1, train, test,cvnt[i], False, RF_error_tot) #Default is 1
        #print(ann1.meanacc)
    
    print("Artificial Neural network: ")
    print("Number of hidden nodes: {}".format(cvhl[i]))
    print("Average Training Accuracy: {}".format(ANN_error_tot[0]/10))
    print("Average Testing Accuracy: {}".format(ANN_error_tot[1]/10))
    print("---------------------")
    print("Random Forest Classifier: ")
    print("Number of Trees: {}".format(cvnt[i]))
    print("Average Training Accuracy: {}".format(RF_error_tot[0]/10))
    print("Average Testing Accuracy: {}".format(RF_error_tot[1]/10))
    print("---------------------")
    ANN_error_tot[0] = 0 
    ANN_error_tot[1] = 0
    RF_error_tot[0] = 0
    RF_error_tot[1] = 0
    
    
#Cross-Validation methods:
    #Test Set Method
    #LOOCV (Leave-One-Out Cross Validation)
    #K-Fold Cross Validation
    #n-Fold Cross Validation
    

#test = pd.DataFrame(nuclear_inp.iloc[:,0:1].values)
#stat = nuclear_inp.iloc[:,0].values
#stat_cat = pd.Categorical(stat)
#power_sens_1 = nuclear_inp.iloc[:,1].values
#power_sens_2
#power_sens_3
#power_sens_4
#press_sens_1 = pd.DataFrame(nuclear_inp.iloc[:,5].values)
#press_sens_2
#press_sens_3
#press_sens_4
#tempe_sens_1
#tempe_sens_2
#tempe_sens_3
#tempe_sens_4
#print (stat.shape)
#print(power_sens_1.shape)
#test = np.concatenate(stat, power_sens_1)

#https://www.cs.toronto.edu/~jlucas/teaching/csc411/lectures/tut5_handout.pdf    
#https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

Data Summary:
Empty values: False.
Shape of the data: columns = 13, rows = 996
Categorical Features: ['Status']
-- Initialising ANN --
Artificial Neural network: 
Number of hidden nodes: 100
Training Accuracy: 0.921875
Testing Accuracy: 0.8
Confusion Matrix: [33  9] , [11 47]
-- Initialising Random Forest --
Random Forest Classifier: 
Min number of samples for leaf node: 1
Training Accuracy: 1.0
Testing Accuracy: 0.88
Confusion Matrix: [39  3] , [ 9 49]
---------------------
Random Forest Classifier: 
Min number of samples for leaf node: 10
Training Accuracy: 0.9453125
Testing Accuracy: 0.88
Confusion Matrix: [38  4] , [ 8 50]
---------------------
Cross Validation
---------------------
Artificial Neural network: 
Number of hidden nodes: 25
Average Training Accuracy: 0.920798868251
Average Testing Accuracy: 0.959434343434
---------------------
Random Forest Classifier: 
Number of Trees: 10
Average Training Accuracy: 0.995648814501
Average Testing Accuracy: 0.908646464646
--------------

In [3]:
nuclear_inp_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Status,996.0,0.500000,0.500251,0.000000,0.000000,0.500000,1.000000,1.000000
Power_range_sensor_1,996.0,4.996993,2.762409,0.008200,2.892120,4.881100,6.775377,12.129800
Power_range_sensor_2,996.0,6.378542,2.313596,0.040300,4.931750,6.470500,8.104500,11.928400
Power_range_sensor_3,996.0,9.227265,2.532658,2.583966,7.500425,9.348000,11.046800,15.759900
Power_range_sensor_4,996.0,7.354094,4.356061,0.062300,3.438141,7.071550,10.917400,17.235858
Pressure _sensor_1,996.0,14.199127,11.680045,0.024800,5.014875,11.716802,20.280250,67.979400
Pressure _sensor_2,996.0,3.077681,2.126752,0.008262,1.412600,2.674746,4.502500,10.242738
Pressure _sensor_3,996.0,5.748279,2.526864,0.001224,4.003975,5.741357,7.503578,12.647500
Pressure _sensor_4,996.0,4.997002,4.165490,0.005800,1.581625,3.859200,7.599900,16.555620
Temperature_sensor_1,996.0,8.155479,6.174639,0.000000,3.190292,6.734450,11.246400,36.186438


In [4]:
#(c,r) = summary(nuclear_inp_df)
#CrossVal(KFolds, nuclear_inp_df)